# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document

**Name**: Aiden Kaminsky


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [34]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------

# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------

# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------

# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [35]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [36]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [37]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [38]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
#!pip install hebrew_tokenizer
# runed the code

In [39]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [40]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [41]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [42]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [43]:
# YOUR CODE HERE
# Add as many code cells as you need
# YOUR CODE HERE

#### Imports

In [44]:
# my imports *no new libbreis
# from sklearn
from sklearn.ensemble import StackingClassifier ,AdaBoostClassifier , BaggingClassifier
from sklearn.model_selection import KFold
from sklearn.feature_selection import r_regression, f_regression, mutual_info_regression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, fbeta_score #to make scoreing easier

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

#### SelectKBest
I am useing **SelectKBest** which is a Univariate **feature selection**. <br>
Univariate feature selection is a way to fillter the features, by selecting <br>
the best features based on univariate statistical tests. <br>
and SelectKBest removes all but the k highest scoring features. <br>

SelectKBest needs a callable which is a univariate statistical tests,  <br>
I took **f_classif**, which Compute the ANOVA F-value for the provided sample.  <br>

ANOVA F-value: The F value is used in analysis of variance (ANOVA). <br>
It is calculated by dividing two mean squares.  <br>
This calculation determines the ratio of explained variance to unexplained variance.  <br>
The F distribution is a theoretical distribution. <br>

In [45]:
print(df_train['gender'].value_counts())

m    575
f    178
Name: gender, dtype: int64


#### Clean text

In [46]:
# Define a function to remove spaces and punctuation marks, etc. from the text
def clean_text(text):
    # remove non-Hebrew words
    text_clean = re.sub(r'[^\u0590-\u05fe]+', ' ', text)
    # remove numbers
    text_clean = re.sub(r'\d+', '', text_clean)
    # Replace all punctuation marks with spaces
    text_clean = re.sub(r'[^\w\s]', ' ', text_clean)
    # Replace all whitespace characters with spaces
    text_clean = re.sub(r'\s+', ' ', text_clean)
    # Strip leading and trailing whitespace
    text_clean = text.strip()
    #remove words longer than num_letters
#     num_letters = 9
#     text_clean = ' '.join([word for word in text_clean.split() if len(word) <= num_letters])
    # remove the letter 'ו' from words that start with 'ו'
    words = text_clean.split()
    for i in range(len(words)):
        if words[i].startswith('ו'):
            words[i] = words[i][1:]
    text_clean = ' '.join(words)
    return text_clean

In [47]:
df = df_train.copy()
text_list = df['story'].tolist()
cleaned_text_list = [clean_text(text) for text in text_list]

Adding a binray gender col, from the string gender col

In [48]:
# change gender col from sting to binray
def y_from_str_to_binary(data):
    # create a new column 'is_male' with binary values 1 if male, 0 if female
    data['is_male'] = (data['gender'] == 'm').astype(int)
    return data

#### Vectorizer

In [49]:
# TfidfVectorizer
X_train = cleaned_text_list
vectorizer_idf = TfidfVectorizer(min_df = 5, ngram_range = (1,1))
X_train_idf = vectorizer_idf.fit_transform(X_train)

Selector

In [50]:
# SelectKBest
selector = SelectKBest(f_classif, k = 300)

In [51]:
# - run loop: GridSearchCV and fit, saving the bast params
def fit_train_models(model_list, all_params, X_train, y_train):
    best_params_all_models = {}
    scores = {}
    best_estimator = {}
    
    costom_score = make_scorer(f1_score, average = 'macro')
    cv = KFold(n_splits = 10, shuffle = True, random_state = 42)
    
    for model_name, model in models_list.items():
        print('-----',model_name,'-----')
        clf = GridSearchCV(model, param_grid= all_params[model_name], cv=cv, scoring=costom_score)
        try:
            clf.fit(X_train, y_train)
        except Exception as e:
            print('error: ', e)
            continue
        # save the best parameters
        best_params_all_models[model_name] = clf.best_params_
        scores[model_name] = clf.best_score_
        best_estimator[model_name] = clf.best_estimator_
        
        print("best_params:", clf.best_params_)
        print("best_score_:", clf.best_score_)
        print()
    return best_params_all_models, scores, best_estimator

The models I tried to use and their params

In [52]:
# list of models
models_list = {
    'Naive_Bayes': MultinomialNB(),
    'Randomf': RandomForestClassifier(),
    'KNN': KNeighborsClassifier(),
    'DecTree': DecisionTreeClassifier(),
    'LinearSVC': LinearSVC(),
    'LogisticReg': LogisticRegression(),
    'Perceptron':Perceptron(),
    'svm': SVC()
              }

# their pramters
# all_params = {
#     'LogisticReg': {'penalty': ['l1', 'l2', 'elasticnet', 'none'],'C': [0.1, 1, 10],
#          'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],'max_iter': [100, 500, 1000]},
#     'KNN': {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']},
#     'DecTree': {'max_depth': [3, 5, 7], 'criterion': ['gini', 'entropy']},
#     'svm': {'C': [0.1, 1, 10],'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],'degree': [2, 3, 4],
#           'gamma': ['scale', 'auto'],'max_iter': [1000]},
#     'LinearSVC': {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2'], 'dual': [False]},
#     #'Randomf': {'n_estimators': [100, 500, 1000],'criterion': ['gini', 'entropy'],
#     #        'max_depth': [None, 5, 10],'min_samples_split': [2, 5, 10],'min_samples_leaf': [1, 2, 4]},
#     'Randomf': {'n_estimators': [100, 200], 'max_depth': [None, 5]},
    # 'Perceptron': {'alpha': [0.0001, 0.05], 'penalty': [None, 'l2', 'l1', 'elasticnet']},
#     'Naive_Bayes': {'alpha': [0.5, 1.0]}
#          }

# selected top params
all_params = {
    'LogisticReg': {'C': [1.0, 10.0], 'solver': ['liblinear']},
    'KNN': {'n_neighbors': [3, 5], 'weights': ['uniform', 'distance']},
    'DecTree': {'max_depth': [3, 5, 7], 'criterion': ['gini', 'entropy']},
    'svm': {'kernel': ['linear', 'rbf'], 'C': [1.0, 10.0]},
    'LinearSVC': {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2'], 'dual': [False]},
    'Naive_Bayes': {'alpha': [0.5, 1.0]},
    'Randomf': {'n_estimators': [100, 200], 'max_depth': [None, 5]},
    'Perceptron': {'alpha': [0.0001, 0.05], 'penalty': [None, 'l2', 'l1', 'elasticnet']}
         }


In [53]:
# Define the gender labels
df = df_train.copy()
df = y_from_str_to_binary(df) # create binary column
y_train = df['is_male'].tolist()
# y_train = df['gender'].tolist() # uncomment to use string column as y

# TfidfVectorizer : X_train_idf
# SelectKBest : X_train_selected
X_train_selected = selector.fit_transform(X_train_idf, y_train)

best_params_all_models, scores, best_estimator = fit_train_models(models_list, all_params, X_train_selected, y_train)


----- Naive_Bayes -----
best_params: {'alpha': 0.5}
best_score_: 0.43285481764217215

----- Randomf -----
best_params: {'max_depth': None, 'n_estimators': 100}
best_score_: 0.6779329266878771

----- KNN -----
best_params: {'n_neighbors': 3, 'weights': 'distance'}
best_score_: 0.4765351290787271

----- DecTree -----
best_params: {'criterion': 'entropy', 'max_depth': 7}
best_score_: 0.5855568708161243

----- LinearSVC -----
best_params: {'C': 100, 'dual': False, 'penalty': 'l2'}
best_score_: 0.8583456560508405

----- LogisticReg -----
best_params: {'C': 10.0, 'solver': 'liblinear'}
best_score_: 0.7383714251857847

----- Perceptron -----
best_params: {'alpha': 0.0001, 'penalty': 'l2'}
best_score_: 0.8451991761517619

----- svm -----
best_params: {'C': 10.0, 'kernel': 'rbf'}
best_score_: 0.8772909557295261



The best model is svm with 0.877 f1_score.

### Predict Text
Useing the model with the highiest score to predict the gender for the stories in test file.

In [54]:
# does the clean and vectorize for the test_stories, and runs the predict
def get_predict_for_test(X_test, vectorizer, model_name, estimator_dict, selector):
    # process the test the same way we did for train
    X_test_clean= [clean_text(text) for text in X_test]
    X_test_vectorize = vectorizer.transform(X_test_clean)
    X_test_kbest = selector.transform(X_test_vectorize)

    predict_result = estimator_dict[model_name].predict(X_test_kbest)
    return predict_result

In [55]:
X_df_test = df_test['story']
predict_result = get_predict_for_test(X_df_test, vectorizer_idf, 'svm', best_estimator, selector)

# # print results
# # 1 - male, 0 - female
count_each = np.unique(predict_result, return_counts=True)
print('Over all boys and girls:')
print('male:',count_each[1][1])
print('female:',count_each[1][0])

Over all boys and girls:
male: 255
female: 68


#### First and Last 5

In [56]:
df_result = df_test.copy()
df_result['predicted_category'] = predict_result
df_result['predicted_category'] = df_result['predicted_category'].map({0: 'f', 1: 'm'})
df_result = df_result.drop(['story'], axis=1)
print('First and Last 5:')
print('First 5:',df_result[0:5])
print('Last 5:',df_result[-5:])

First and Last 5:
First 5:    test_example_id predicted_category
0                0                  m
1                1                  m
2                2                  m
3                3                  m
4                4                  m
Last 5:      test_example_id predicted_category
318              318                  m
319              319                  m
320              320                  m
321              321                  m
322              322                  m


In [57]:
df_result

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m
...,...,...
318,318,m
319,319,m
320,320,m
321,321,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [60]:
df_predicted.to_csv('classification_results.csv',index=False)

NameError: name 'df_predicted' is not defined